In [1]:
#this stuff is used in the confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#image display thing
from PIL import Image

#basic library imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#tf imports
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import*
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

import os


In [ ]:
#this code finds the kaggle dataset and places the kaggle API in the correct directory
#make sure that kaggle API token is put into project at runtime
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle -v

!kaggle datasets download -d vencerlanz09/sea-animals-image-dataste

!unzip sea-animals-image-dataste.zip

In [14]:
# Define shape of data, batch size,
# and amount of epochs
INPUT_SHAPE = (224,224,3)
BATCH_SIZE = 32
EPOCHS = 50

In [5]:
#Split data set into 80% training and 20% validation
datagen = ImageDataGenerator(
            rotation_range=20,
            horizontal_flip=True,
            vertical_flip=True,
            rescale=1./255,
            validation_split=0.2
                )
train_datagen = datagen.flow_from_directory(
                '/content/Data',
                target_size=(INPUT_SHAPE[0],INPUT_SHAPE[1]),
                class_mode='categorical',
                batch_size=BATCH_SIZE,
                shuffle=True,
                subset = 'training'
    )
test_datagen = datagen.flow_from_directory(
                '/content/Data',
                target_size=(INPUT_SHAPE[0],INPUT_SHAPE[1]),
                class_mode='categorical',
                batch_size=BATCH_SIZE,
                shuffle=True,
                subset = 'validation'
)

Found 10979 images belonging to 23 classes.
Found 2732 images belonging to 23 classes.


In [6]:
# dataset_path = 'content/data'

# # Get list of image file names
# image_files = os.listdir(dataset_path)

# # Select 12 random indices
# random_indices = np.random.choice(len(image_files), size=12, replace=False)

# # Plot images
# plt.figure(figsize=(12, 8))  # Adjust size of the plot as needed
# for i, idx in enumerate(random_indices):
#     image_path = os.path.join(dataset_path, image_files[idx])
#     image = Image.open(image_path)
#     plt.subplot(3, 4, i + 1)  # 3 rows, 4 columns, position i+1
#     plt.imshow(image)
#     plt.title(f"Image {idx}")
#     plt.axis('off')  # Hide axes
# plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'content/data'

In [7]:
#define the architechture of the neural network model

#this model uses VGG16
vgg_model = VGG16(weights = 'imagenet',include_top=False, input_shape = INPUT_SHAPE)
for layers in vgg_model.layers:
    layers.trainable = False

inputs = Input(INPUT_SHAPE)
vgg_output = vgg_model(inputs)


x = Flatten()(vgg_output)
x = Dense(512)(x)
x = Dense(256)(x)
x = Dense(128)(x)
x = Dense(23)(x)
model = Model(inputs = [inputs],outputs = [x])

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
#train the model
model.compile(optimizer=Adam(),
             loss=CategoricalCrossentropy(from_logits=True),
             metrics=['accuracy','Recall'])
history = model.fit(train_datagen,validation_data=test_datagen,epochs=EPOCHS)

Epoch 1/50
344/344 [==============================] - 184s 531ms/step - loss: 1.8820 - accuracy: 0.5401 - recall: 0.9574 - val_loss: 1.7862 - val_accuracy: 0.5168 - val_recall: 0.9649
Epoch 2/50
344/344 [==============================] - 182s 530ms/step - loss: 1.6197 - accuracy: 0.5681 - recall: 0.9651 - val_loss: 2.1219 - val_accuracy: 0.5037 - val_recall: 0.9484
Epoch 3/50
344/344 [==============================] - 181s 525ms/step - loss: 1.5059 - accuracy: 0.5919 - recall: 0.9709 - val_loss: 1.7851 - val_accuracy: 0.5179 - val_recall: 0.9257
Epoch 4/50
344/344 [==============================] - 181s 527ms/step - loss: 1.4645 - accuracy: 0.5981 - recall: 0.9711 - val_loss: 2.0102 - val_accuracy: 0.5216 - val_recall: 0.9524
Epoch 5/50
344/344 [==============================] - 184s 534ms/step - loss: 1.4453 - accuracy: 0.5979 - recall: 0.9735 - val_loss: 1.7430 - val_accuracy: 0.5560 - val_recall: 0.9623
Epoch 6/50
344/344 [==============================] - 180s 522ms/step - loss: 1.

In [1]:
#evaluate the model
model.evaluate(test_datagen)

NameError: name 'model' is not defined

In [2]:
#plot the accuracy of the model as a function of the epoch
import matplotlib.pyplot as plt


def plot_accuracy(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

# Assuming 'history' contains the training history of your model

# Plot accuracy
plot_accuracy(history)


NameError: name 'history' is not defined

In [13]:
true_label = []
predicted_label = []

for images, labels in test_ds_preprocessed.unbatch():
    # Store true labels (Convert one-hot to index)
    true_label = np.argmax(labels.numpy())
    true_labels.append(true_label)

    # Get model prediction (Predict expects batch dimension)
    pred = model.predict(tf.expand_dims(images, 0), verbose=False)
    predicted_label = np.argmax(pred)
    predicted_labels.append(predicted_label)

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_label, predicted_label)

# Plot confusion matrix
plt.figure(figsize=(10, 8))  # Adjust size of the plot as needed
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=range(23), yticklabels=range(23))
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

NameError: name 'test_ds_preprocessed' is not defined